In [ ]:
# Uncomment to install/upgrade ras-commander from pip
#!pip install --upgrade ras-commander

#Import the ras-commander package
from ras_commander import *

##### Optional Code Cell For Development/Testing Mode (Local Copy)
##### Uncomment and run this cell instead of the pip cell above

```python
# For Development Mode, add the parent directory to the Python path
import os
import sys
from pathlib import Path

current_file = Path(os.getcwd()).resolve()
rascmdr_directory = current_file.parent

# Use insert(0) instead of append() to give highest priority to local version
if str(rascmdr_directory) not in sys.path:
    sys.path.insert(0, str(rascmdr_directory))

print("Loading ras-commander from local dev copy")
from ras_commander import *
```

In [ ]:
import sys
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from shapely.geometry import Point
import math

In [ ]:
def create_manning_minmax_df():
    """
    Create a dataframe containing minimum and maximum Manning's n values
    based on recommended ranges from literature.
    
    Returns:
        pd.DataFrame: DataFrame with columns for Land Cover Name, min_n, max_n
    """
    # Define the data as a list of dictionaries
    manning_data = [
        {"Land Cover Name": "NoData", "min_n": 0.050, "max_n": 0.070},
        {"Land Cover Name": "Barren Land Rock/Sand/Clay", "min_n": 0.023, "max_n": 0.100},
        {"Land Cover Name": "Cultivated Crops", "min_n": 0.020, "max_n": 0.100},
        {"Land Cover Name": "Deciduous Forest", "min_n": 0.100, "max_n": 0.200},
        {"Land Cover Name": "Developed, High Intensity", "min_n": 0.120, "max_n": 0.200},
        {"Land Cover Name": "Developed, Low Intensity", "min_n": 0.060, "max_n": 0.120},
        {"Land Cover Name": "Developed, Medium Intensity", "min_n": 0.080, "max_n": 0.160},
        {"Land Cover Name": "Developed, Open Space", "min_n": 0.030, "max_n": 0.090},
        {"Land Cover Name": "Emergent Herbaceous Wetlands", "min_n": 0.050, "max_n": 0.120},
        {"Land Cover Name": "Evergreen Forest", "min_n": 0.080, "max_n": 0.160},
        {"Land Cover Name": "Grassland/Herbaceous", "min_n": 0.025, "max_n": 0.070},
        {"Land Cover Name": "Mixed Forest", "min_n": 0.080, "max_n": 0.200},
        {"Land Cover Name": "Open Water", "min_n": 0.025, "max_n": 0.050},
        {"Land Cover Name": "Pasture/Hay", "min_n": 0.025, "max_n": 0.090},
        {"Land Cover Name": "Shrub/Scrub", "min_n": 0.070, "max_n": 0.160},
        {"Land Cover Name": "Woody Wetlands", "min_n": 0.045, "max_n": 0.150}
    ]
    
    # Create DataFrame
    df = pd.DataFrame(manning_data)
    
    # Calculate the midpoint value
    df['mid_n'] = (df['min_n'] + df['max_n']) / 2
    
    # Sort by land cover name
    df = df.sort_values('Land Cover Name').reset_index(drop=True)
    
    # Print summary information
    print(f"Manning's n value ranges for {len(df)} land cover types:")
    print(df)
    
    return df

# Create the Manning's n ranges dataframe
manning_minmax_df = create_manning_minmax_df()

In [4]:
def analyze_mesh_land_cover_statistics(project_folder, geom_number=None, plan_number=None):
    """
    Analyze the land cover statistics for a 2D mesh area in a HEC-RAS model,
    excluding areas controlled by regional Manning's n overrides.
    
    Args:
        project_folder (str): Path to the HEC-RAS project folder
        geom_number (str, optional): Geometry number to use. If None, will use
                                    geometry from plan_number or the first geometry.
        plan_number (str, optional): Plan number to use. If None, will use the first plan.
    
    Returns:
        pd.DataFrame: DataFrame with land cover statistics for areas controlled by base overrides
    """
    # Initialize RAS project
    ras = init_ras_project(project_folder, "6.6")
    
    # [existing code to get geometry number and paths]
    
    # Get the geometry file path
    geom_path = ras.geom_df.loc[ras.geom_df['geom_number'] == geom_number, 'full_path'].values[0]
    
    # Get the geometry HDF path
    geom_hdf_path = ras.geom_df.loc[ras.geom_df['geom_number'] == geom_number, 'hdf_path'].values[0]
    
    # Get mesh areas from the geometry
    mesh_areas_gdf = HdfMesh.get_mesh_areas(geom_hdf_path)
    num_mesh_areas = len(mesh_areas_gdf)
    
    # Get the base Manning's overrides to compare with land cover statistics
    base_overrides = RasGeo.get_mannings_baseoverrides(geom_path)
    
    # Get regional override information
    region_overrides = RasGeo.get_mannings_regionoverrides(geom_path)
    regional_mask = None
    
    # If regional overrides exist, get their geometries to exclude them
    if not region_overrides.empty:
        print("Regional Manning's n overrides found - these areas will be excluded from base sensitivity analysis")
        # Get regional override polygons from the geometry
        regional_polygons_gdf = get_regional_override_polygons(geom_hdf_path)
        
        if not regional_polygons_gdf.empty:
            # Create a union of all regional override polygons to use as a mask
            regional_mask = regional_polygons_gdf.unary_union
            print(f"Excluding {len(regional_polygons_gdf)} regional override areas from analysis")
    
    all_results = {}
    
    for idx, row in mesh_areas_gdf.iterrows():
        mesh_name = row['mesh_name']
        mesh_geom = row['geometry']
        
        print(f"Analyzing land cover for mesh area: {mesh_name}")
        
        # Get effective mesh area (excluding regional overrides)
        effective_mesh_geom = mesh_geom
        if regional_mask is not None:
            if mesh_geom.intersects(regional_mask):
                effective_mesh_geom = mesh_geom.difference(regional_mask)
                print(f"  Excluded regional override areas from mesh {mesh_name}")
        
        total_area = effective_mesh_geom.area
        
        # Create a simulated land cover distribution based on base_overrides
        # In reality, you would use actual spatial analysis with the land cover raster
        landcover_stats = []
        
        # Use the land cover types from the base overrides
        for _, override_row in base_overrides.iterrows():
            land_cover = override_row['Land Cover Name']
            n_value = override_row["Base Manning's n Value"]
            
            # Generate a random percentage for this example
            # In reality, this would come from actual spatial analysis
            np.random.seed(hash(land_cover) % 2**32)  # Use the land cover name as a seed
            percentage = np.random.random() * 25  # Random percentage between 0-25%
            
            area = total_area * (percentage / 100)
            
            landcover_stats.append({
                'Land Cover Type': land_cover,
                'Area': area,
                'Percentage': percentage,
                'Current_n': n_value
            })
        
        # Create DataFrame and sort by percentage
        landcover_df = pd.DataFrame(landcover_stats)
        landcover_df = landcover_df.sort_values('Percentage', ascending=False).reset_index(drop=True)
        
        # Store the results
        all_results[mesh_name] = landcover_df
    
    # If there's only one mesh area, return its dataframe directly
    if len(all_results) == 1:
        return next(iter(all_results.values()))
    
    return all_results

def get_regional_override_polygons(geom_hdf_path):
    """
    Extract regional override polygon geometries from a HEC-RAS geometry HDF file.
    
    Args:
        geom_hdf_path (str): Path to the HEC-RAS geometry HDF file
        
    Returns:
        geopandas.GeoDataFrame: GeoDataFrame with regional override polygons
    """
    import h5py
    import geopandas as gpd
    from shapely.geometry import Polygon
    
    try:
        with h5py.File(geom_hdf_path, 'r') as f:
            # Navigate to regional override polygons in the HDF structure
            # This path would need to be determined based on the HEC-RAS HDF structure
            if 'Geometry/Regional Manning Areas' in f:
                region_group = f['Geometry/Regional Manning Areas']
                
                polygons = []
                region_names = []
                
                # Process each regional override polygon
                for region_name, region_data in region_group.items():
                    # Extract polygon coordinates
                    # This is a simplified example; actual implementation would depend on HDF structure
                    if 'Polygon' in region_data:
                        coords = region_data['Polygon'][:]
                        polygon = Polygon(coords)
                        polygons.append(polygon)
                        region_names.append(region_name)
                
                # Create GeoDataFrame
                if polygons:
                    gdf = gpd.GeoDataFrame(
                        {'region_name': region_names, 'geometry': polygons},
                        crs='EPSG:4326'  # Set appropriate CRS
                    )
                    return gdf
        
        # Return empty GeoDataFrame if no regional overrides found
        return gpd.GeoDataFrame(columns=['region_name', 'geometry'])
        
    except Exception as e:
        print(f"Error extracting regional override polygons: {str(e)}")
        return gpd.GeoDataFrame(columns=['region_name', 'geometry'])

In [5]:
def generate_sensitivity_values(min_val, max_val, current_val, interval=0.01):
    """
    Generate a list of Manning's n values for sensitivity testing.
    
    Args:
        min_val (float): Minimum value from literature
        max_val (float): Maximum value from literature
        current_val (float): Current value in the model
        interval (float): Interval between test values
    
    Returns:
        list: List of n values to test
    """
    # Round values to avoid floating point issues
    min_val = round(min_val, 4)
    max_val = round(max_val, 4)
    current_val = round(current_val, 4)
    interval = round(interval, 4)
    
    # Generate values from min to max at specified interval
    all_values = np.arange(min_val, max_val + interval/2, interval)
    all_values = np.round(all_values, 4)  # Round to avoid floating point issues
    
    # Remove current value if it's in the range
    values = [val for val in all_values if abs(val - current_val) > interval/2]
    
    # Make sure current value is not in the list
    if current_val in values:
        values.remove(current_val)
    
    return values

def estimate_plan_count(significant_landuses, n_ranges, interval=0.01):
    """
    Estimate the number of plans that will be created for sensitivity analysis.
    
    Args:
        significant_landuses (pd.DataFrame): DataFrame with significant land cover types
        n_ranges (pd.DataFrame): DataFrame with Manning's n ranges
        interval (float): Interval between test values
    
    Returns:
        int: Estimated number of plans
    """
    total_plans = 0
    
    for _, landuse in significant_landuses.iterrows():
        land_cover = landuse['Land Cover Type']
        current_n = landuse['Current_n']
        
        # Find matching land cover in n_ranges
        match = n_ranges[n_ranges['Land Cover Name'] == land_cover]
        if match.empty:
            continue
            
        min_n = match['min_n'].values[0]
        max_n = match['max_n'].values[0]
        
        # Count values between min and max at interval spacing, excluding current value
        values = generate_sensitivity_values(min_n, max_n, current_n, interval)
        total_plans += len(values)
    
    return total_plans

In [6]:
def individual_landuse_sensitivity_base(
    project_folder,
    template_plan,
    point_of_interest,
    area_threshold=10.0,  # percentage threshold for significant land uses
    interval=0.01,
    max_workers=2,
    num_cores=2,
    output_folder="Individual_Landuse_Sensitivity",
    custom_n_ranges=None  # optional custom Manning's n ranges
):
    """
    Perform sensitivity analysis by varying individual land use Manning's n values
    in the base overrides.
    
    Args:
        project_folder (str): Path to HEC-RAS project folder
        template_plan (str): Plan number to use as template
        point_of_interest (tuple or Point): Coordinates for extracting results
        area_threshold (float): Percentage threshold for significant land uses
        interval (float): Interval for Manning's n test values
        max_workers (int): Number of parallel workers
        num_cores (int): Number of cores per worker
        output_folder (str): Name of output folder
        custom_n_ranges (pd.DataFrame): Optional custom Manning's n ranges
    
    Returns:
        dict: Results of sensitivity analysis
    """
    import time
    from datetime import datetime
    
    # Convert point_of_interest to Point if not already
    if not isinstance(point_of_interest, Point):
        point_of_interest = Point(point_of_interest[0], point_of_interest[1])
    
    # Use default or custom Manning's n ranges
    n_ranges = custom_n_ranges if custom_n_ranges is not None else create_manning_minmax_df()
    
    # Create timestamp for unique run identifier
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Initialize RAS project
    print(f"Initializing HEC-RAS project: {project_folder}")
    ras = init_ras_project(project_folder, "6.6")
    
    # Create output directory
    results_dir = Path(project_folder) / output_folder
    results_dir.mkdir(exist_ok=True)
    print(f"Results will be saved to: {results_dir}")
    
    # Verify template plan exists
    if template_plan not in ras.plan_df['plan_number'].values:
        raise ValueError(f"Template plan {template_plan} not found in project")
    
    # Get the geometry number for the template plan
    template_geom = ras.plan_df.loc[ras.plan_df['plan_number'] == template_plan, 'geometry_number'].values[0]
    print(f"\nTemplate plan: {template_plan} (Geometry: {template_geom})")
    
    # Get the geometry file path
    geom_path = ras.geom_df.loc[ras.geom_df['geom_number'] == template_geom, 'full_path'].values[0]
    
    # Get the original Manning's values
    original_baseoverrides = RasGeo.get_mannings_baseoverrides(geom_path)
    original_regionoverrides = RasGeo.get_mannings_regionoverrides(geom_path)
    
    # Analyze land cover statistics for the 2D mesh areas
    print("\nAnalyzing land cover statistics for the 2D mesh areas...")
    landcover_stats = analyze_mesh_land_cover_statistics(
        project_folder, 
        geom_number=template_geom
    )
    
    if landcover_stats is None:
        raise ValueError("Could not analyze land cover statistics")
    
    # Identify significant land uses (above threshold)
    significant_landuses = landcover_stats[landcover_stats['Percentage'] >= area_threshold].copy()
    significant_landuses = significant_landuses.sort_values('Percentage', ascending=False).reset_index(drop=True)
    
    if len(significant_landuses) == 0:
        print(f"No land uses found with coverage above {area_threshold}% threshold")
        return None
    
    print(f"\nFound {len(significant_landuses)} significant land uses (>= {area_threshold}% coverage):")
    print(significant_landuses[['Land Cover Type', 'Percentage', 'Current_n']])
    
    # Check if we'll exceed the plan limit
    current_plan_count = len(ras.plan_df)
    max_plans = 99  # HEC-RAS limit
    remaining_plans = max_plans - current_plan_count
    
    # Estimate the number of plans needed
    estimated_plan_count = estimate_plan_count(significant_landuses, n_ranges, interval)
    
    if estimated_plan_count > remaining_plans:
        print(f"\nWARNING: This analysis would create approximately {estimated_plan_count} plans, but only {remaining_plans} more plans can be added (limit is 99)")
        print("Consider adjusting the following to reduce the number of plans:")
        print(f"1. Increase the area threshold (currently {area_threshold}%)")
        print(f"2. Increase the interval between test values (currently {interval})")
        print(f"3. Reduce the min/max ranges for land uses")
        print(f"4. Select fewer land uses to test")
        
        # Ask for confirmation to continue
        response = input("\nDo you want to continue anyway? (y/n): ")
        if response.lower() != 'y':
            print("Analysis canceled")
            return None
    
    # Store the current (template) plan as base scenario
    scenarios = [{
        'name': 'Template',
        'plan_number': template_plan,
        'geom_number': template_geom,
        'shortid': 'Template',
        'land_cover': None,
        'n_value': None,
        'description': "Original Manning's n Values"
    }]
    
    # Function to create a modified plan with adjusted Manning's values for a specific land use
    def create_modified_plan(land_cover, new_n_value):
        # Create a shortid based on land cover and n value
        # Convert land cover name to code (e.g. "Open Water" -> "OW")
        code = ''.join([word[0] for word in land_cover.split() if word[0].isalpha()])
        if not code:
            code = land_cover[:2]
        code = code.upper()
        
        # Format n value for shortid
        n_str = f"{new_n_value:.3f}".replace(".", "")
        shortid = f"B_{code}_{n_str}"
        
        print(f"\nCreating plan for '{land_cover}' with n = {new_n_value} (ShortID: {shortid})")
        
        # Clone the template plan
        new_plan_number = RasPlan.clone_plan(template_plan, new_plan_shortid=shortid)
        
        # Clone the template geometry
        new_geom_number = RasPlan.clone_geom(template_geom)
        
        # Set the new plan to use the new geometry
        RasPlan.set_geom(new_plan_number, new_geom_number)
        
        # Get the new geometry file path
        new_geom_path = ras.geom_df.loc[ras.geom_df['geom_number'] == new_geom_number, 'full_path'].values[0]
        
        # Create modified base overrides
        modified_baseoverrides = original_baseoverrides.copy()
        
        # Update the Manning's n value for this specific land cover type
        land_cover_mask = modified_baseoverrides['Land Cover Name'] == land_cover
        if land_cover_mask.any():
            current_n = modified_baseoverrides.loc[land_cover_mask, "Base Manning's n Value"].values[0]
            print(f"  Changing '{land_cover}' from {current_n:.4f} to {new_n_value:.4f}")
            modified_baseoverrides.loc[land_cover_mask, "Base Manning's n Value"] = new_n_value
        else:
            print(f"  Warning: Land cover '{land_cover}' not found in base overrides")
        
        # Apply the modified base overrides
        RasGeo.set_mannings_baseoverrides(new_geom_path, modified_baseoverrides)
        
        # Copy regional overrides unchanged if they exist
        if not original_regionoverrides.empty:
            RasGeo.set_mannings_regionoverrides(new_geom_path, original_regionoverrides)
        
        # Store scenario details
        return {
            'name': f"{land_cover}_{new_n_value:.3f}",
            'plan_number': new_plan_number,
            'geom_number': new_geom_number,
            'shortid': shortid,
            'land_cover': land_cover,
            'n_value': new_n_value,
            'description': f"Manning's n = {new_n_value:.3f} for {land_cover}"
        }
    
    # Create plans for each significant land use with varying n values
    all_plans_to_run = []
    
    for _, landuse in significant_landuses.iterrows():
        land_cover = landuse['Land Cover Type']
        current_n = landuse['Current_n']
        
        # Find matching land cover in n_ranges
        match = n_ranges[n_ranges['Land Cover Name'] == land_cover]
        
        if match.empty:
            print(f"Warning: No Manning's n range found for '{land_cover}'. Skipping.")
            continue
            
        min_n = match['min_n'].values[0]
        max_n = match['max_n'].values[0]
        
        print(f"\nProcessing land cover: {land_cover}")
        print(f"  Current n: {current_n:.4f}")
        print(f"  Literature range: {min_n:.4f} to {max_n:.4f}")
        
        # Generate test values within the range, excluding the current value
        test_values = generate_sensitivity_values(min_n, max_n, current_n, interval)
        
        print(f"  Testing {len(test_values)} values: {[round(val, 3) for val in test_values]}")
        
        # Create a plan for each test value
        for n_value in test_values:
            new_scenario = create_modified_plan(land_cover, n_value)
            scenarios.append(new_scenario)
            all_plans_to_run.append(new_scenario['plan_number'])
    
    # Save scenario information
    scenario_info = pd.DataFrame(scenarios)
    scenario_info_path = results_dir / "scenarios.csv"
    scenario_info.to_csv(scenario_info_path, index=False)
    print(f"\nScenario information saved to: {scenario_info_path}")
    
    # Run the plans (excluding the template which is already computed)
    plans_to_run = [plan for plan in all_plans_to_run if plan != template_plan]
    
    if not plans_to_run:
        print("No plans to run.")
        return {'scenarios': scenarios, 'output_folder': results_dir}
    
    print(f"\nRunning {len(plans_to_run)} plans in parallel...")
    execution_results = RasCmdr.compute_parallel(
        plan_number=plans_to_run,
        max_workers=max_workers,
        num_cores=num_cores,
        clear_geompre=True
    )
    
    print("\nExecution results:")
    for plan, success in execution_results.items():
        print(f"  Plan {plan}: {'Successful' if success else 'Failed'}")
    
    # If point of interest provided, extract and compare results
    if point_of_interest is not None:
        # Get geometry HDF path for cell identification
        geom_hdf_path = ras.geom_df.loc[ras.geom_df['geom_number'] == template_geom, 'hdf_path'].values[0]
        
        # Find the nearest mesh cell
        mesh_cells_gdf = HdfMesh.get_mesh_cell_points(geom_hdf_path)
        distances = mesh_cells_gdf.geometry.apply(lambda geom: geom.distance(point_of_interest))
        nearest_idx = distances.idxmin()
        mesh_cell_id = mesh_cells_gdf.loc[nearest_idx, 'cell_id']
        mesh_name = mesh_cells_gdf.loc[nearest_idx, 'mesh_name']
        
        print(f"\nNearest cell ID: {mesh_cell_id}")
        print(f"Distance: {distances[nearest_idx]:.2f} units")
        print(f"Mesh area: {mesh_name}")
        
        # Extract results for each scenario
        all_results = {}
        max_ws_values = []
        
        for scenario in scenarios:
            plan_number = scenario['plan_number']
            land_cover = scenario['land_cover']
            n_value = scenario['n_value']
            shortid = scenario['shortid']
            
            try:
                results_xr = HdfResultsMesh.get_mesh_cells_timeseries(plan_number)
                
                # Extract water surface data
                ws_data = results_xr[mesh_name]['Water Surface'].sel(cell_id=int(mesh_cell_id))
                
                # Convert to DataFrame
                ws_df = pd.DataFrame({
                    'time': ws_data.time.values,
                    'water_surface': ws_data.values
                })
                
                # Store results
                max_ws = ws_df['water_surface'].max()
                
                all_results[plan_number] = {
                    'scenario': scenario,
                    'df': ws_df,
                    'max_water_surface': max_ws
                }
                
                max_ws_values.append({
                    'plan_number': plan_number,
                    'shortid': shortid,
                    'land_cover': land_cover,
                    'n_value': n_value,
                    'max_water_surface': max_ws
                })
                
                print(f"  {shortid}: Max WSE = {max_ws:.2f}")
                
                # Save time series to CSV
                ws_df.to_csv(results_dir / f"timeseries_{shortid}.csv", index=False)
                
            except Exception as e:
                print(f"  Error extracting results for {shortid}: {str(e)}")
        
        # Create summary DataFrame
        if max_ws_values:
            max_ws_df = pd.DataFrame(max_ws_values)
            max_ws_df.to_csv(results_dir / "max_water_surface_summary.csv", index=False)
            
            # Create plots by land cover type
            for land_cover in significant_landuses['Land Cover Type']:
                # Filter scenarios for this land cover
                land_cover_scenarios = max_ws_df[max_ws_df['land_cover'] == land_cover].copy()
                
                # Add the template scenario
                template_row = max_ws_df[max_ws_df['shortid'] == 'Template']
                if not template_row.empty:
                    land_cover_scenarios = pd.concat([template_row, land_cover_scenarios])
                
                if land_cover_scenarios.empty:
                    continue
                
                # Sort by n_value
                land_cover_scenarios = land_cover_scenarios.sort_values('n_value').reset_index(drop=True)
                
                # Create plot
                fig, ax = plt.subplots(figsize=(10, 6))
                ax.plot(land_cover_scenarios['n_value'], land_cover_scenarios['max_water_surface'], 
                         marker='o', linestyle='-', linewidth=2)
                
                # Add template point in a different color if it exists
                template_idx = land_cover_scenarios[land_cover_scenarios['shortid'] == 'Template'].index
                if not template_idx.empty:
                    ax.scatter(land_cover_scenarios.loc[template_idx, 'n_value'], 
                                land_cover_scenarios.loc[template_idx, 'max_water_surface'],
                                color='red', s=100, zorder=5, label='Template')
                
                # Add labels and title
                ax.set_xlabel(f"Manning's n for {land_cover}")
                ax.set_ylabel("Maximum Water Surface Elevation (ft)")
                ax.set_title(f"Sensitivity to {land_cover} Manning's n Value")
                ax.grid(True, linestyle='--', alpha=0.7)
                
                if not template_idx.empty:
                    ax.legend()
                
                # Save plot
                plot_path = results_dir / f"sensitivity_{land_cover.replace(' ', '_').replace('/', '_')}.png"
                plt.tight_layout()
                plt.savefig(plot_path)
                plt.close()
                print(f"Created sensitivity plot for {land_cover}")
            
            # Create time series comparison plot for each land cover
            for land_cover in significant_landuses['Land Cover Type']:
                fig, ax = plt.subplots(figsize=(12, 6))
                
                # Get template results
                template_plan = scenarios[0]['plan_number']
                if template_plan in all_results:
                    template_df = all_results[template_plan]['df']
                    ax.plot(template_df['time'], template_df['water_surface'], 
                             color='black', linewidth=2, label='Template')
                
                # Filter scenarios for this land cover and plot
                land_cover_scenarios = [s for s in scenarios if s['land_cover'] == land_cover]
                
                if not land_cover_scenarios:
                    plt.close()
                    continue
                
                # Setup colormap for n values
                n_values = [s['n_value'] for s in land_cover_scenarios if s['n_value'] is not None]
                if not n_values:
                    plt.close()
                    continue
                    
                min_n = min(n_values)
                max_n = max(n_values)
                norm = plt.Normalize(min_n, max_n)
                cmap = plt.cm.viridis
                
                # Plot each scenario with explicit legend entries
                for scenario in land_cover_scenarios:
                    plan_number = scenario['plan_number']
                    n_value = scenario['n_value']
                    
                    if plan_number in all_results and n_value is not None:
                        df = all_results[plan_number]['df']
                        color = cmap(norm(n_value))
                        label = f"{land_cover}: n = {n_value:.3f}"
                        ax.plot(df['time'], df['water_surface'], color=color, 
                                 linewidth=1, alpha=0.7, label=label)
                
                # Add labels and title
                ax.set_xlabel("Time")
                ax.set_ylabel("Water Surface Elevation (ft)")
                ax.set_title(f"WSE Time Series for Different {land_cover} Manning's n Values")
                ax.grid(True, linestyle='--', alpha=0.7)
                
                # Add legend with land cover and n values
                ax.legend(loc='best', fontsize='small', title="Scenarios")
                
                # Add colorbar
                sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
                sm.set_array([])
                plt.colorbar(sm, ax=ax).set_label(f"Manning's n for {land_cover}")
                
                # Save plot
                plot_path = results_dir / f"timeseries_{land_cover.replace(' ', '_').replace('/', '_')}.png"
                plt.tight_layout()
                plt.savefig(plot_path)
                plt.close()
                print(f"Created time series plot for {land_cover}")
    
    # Return results
    return {
        'scenarios': scenarios,
        'execution_results': execution_results if 'execution_results' in locals() else None,
        'results': all_results if 'all_results' in locals() else None,
        'max_ws_summary': max_ws_df if 'max_ws_df' in locals() else None,
        'significant_landuses': significant_landuses,
        'output_folder': results_dir
    }

In [ ]:
# Example usage for Base Overrides Sensitivity Analysis
# To run this, uncomment the code, adjust parameters as needed, and execute the cell
# Define project path and template plan
project_folder = Path(os.getcwd()) / "example_projects" / "BaldEagleCrkMulti2D"
template_plan = "03"  # Use plan 03 as the template


# Define a point of interest for result extraction
point_of_interest = (2081544, 365715)  # Coordinates where you want to extract results

# Extract and prepare the example project
RasExamples.extract_project(["BaldEagleCrkMulti2D"])

# Define project path and template plan
project_folder = Path(os.getcwd()) / "example_projects" / "BaldEagleCrkMulti2D"
template_plan = "03"  # Use plan 03 as the template

# Define a point of interest for result extraction
point_of_interest = (2081544, 365715)  # Coordinates where you want to extract results

In [ ]:
'''

# Run the base sensitivity analysis
base_sensitivity_results = individual_landuse_sensitivity_base(
    project_folder=project_folder,
    template_plan=template_plan,
    point_of_interest=point_of_interest,
    area_threshold=15.0,  # Only analyze land uses covering at least 10% of the mesh area
    interval=0.02,       # Adjust interval to reduce the number of test values
    max_workers=2,
    num_cores=2,
    output_folder="Base_Landuse_Sensitivity"
)



# Print summary information
if base_sensitivity_results:
    print("\nAnalysis complete! Results saved to:", base_sensitivity_results['output_folder'])
    if 'significant_landuses' in base_sensitivity_results:
        print("\nSignificant land uses analyzed:")
        print(base_sensitivity_results['significant_landuses'][['Land Cover Type', 'Percentage']])

'''

-----

# Base Overrides Sensitivity Results from the HEC Example Project BaldEagleCrkMulti2D, Plan 03: 

<!-- Barren Land/Rock/Sand/Clay -->
![Time Series - Barren Land/Rock/Sand/Clay](data/manning_img/timeseries_Barren_Land_Rock_Sand_Clay.png)
![Mannings n Sensitivity - Barren Land/Rock/Sand/Clay](data/manning_img/sensitivity_Barren_Land_Rock_Sand_Clay.png)

<!-- Cultivated Crops -->
![Time Series - Cultivated Crops](data/manning_img/timeseries_Cultivated_Crops.png)
![Mannings n Sensitivity - Cultivated Crops](data/manning_img/sensitivity_Cultivated_Crops.png)

<!-- Deciduous Forest -->
![Time Series - Deciduous Forest](data/manning_img/timeseries_Deciduous_Forest.png)
![Mannings n Sensitivity - Deciduous Forest](data/manning_img/sensitivity_Deciduous_Forest.png)

<!-- Developed High Intensity -->
![Time Series - Developed High Intensity](data/manning_img/timeseries_Developed_High_Intensity.png)
![Mannings n Sensitivity - Developed High Intensity](data/manning_img/sensitivity_Developed_High_Intensity.png)

<!-- Developed Low Intensity -->
![Time Series - Developed Low Intensity](data/manning_img/timeseries_Developed_Low_Intensity.png)
![Mannings n Sensitivity - Developed Low Intensity](data/manning_img/sensitivity_Developed_Low_Intensity.png)

<!-- Developed Medium Intensity -->
![Time Series - Developed Medium Intensity](data/manning_img/timeseries_Developed_Medium_Intensity.png)
![Mannings n Sensitivity - Developed Medium Intensity](data/manning_img/sensitivity_Developed_Medium_Intensity.png)

<!-- Developed Open Space -->
![Time Series - Developed Open Space](data/manning_img/timeseries_Developed_Open_Space.png)
![Mannings n Sensitivity - Developed Open Space](data/manning_img/sensitivity_Developed_Open_Space.png)

<!-- Emergent Herbaceous Wetlands -->
![Time Series - Emergent Herbaceous Wetlands](data/manning_img/timeseries_Emergent_Herbaceous_Wetlands.png)
![Mannings n Sensitivity - Emergent Herbaceous Wetlands](data/manning_img/sensitivity_Emergent_Herbaceous_Wetlands.png)

-----

# Example: Sensitivity for Regional Overrides:

In [9]:
def individual_landuse_sensitivity_region(
    project_folder,
    template_plan,
    point_of_interest,
    area_threshold=10.0,  # percentage threshold for significant land uses
    interval=0.01,
    max_workers=2,
    num_cores=2,
    region_name=None,  # optional specific region to analyze
    output_folder="Regional_Landuse_Sensitivity",
    custom_n_ranges=None  # optional custom Manning's n ranges
):
    """
    Perform sensitivity analysis by varying individual land use Manning's n values
    in the regional overrides.
    
    Args:
        project_folder (str): Path to HEC-RAS project folder
        template_plan (str): Plan number to use as template
        point_of_interest (tuple or Point): Coordinates for extracting results
        area_threshold (float): Percentage threshold for significant land uses
        interval (float): Interval for Manning's n test values
        max_workers (int): Number of parallel workers
        num_cores (int): Number of cores per worker
        region_name (str): Optional specific region to analyze
        output_folder (str): Name of output folder
        custom_n_ranges (pd.DataFrame): Optional custom Manning's n ranges
    
    Returns:
        dict: Results of sensitivity analysis
    """
    import time
    from datetime import datetime
    
    # Convert point_of_interest to Point if not already
    if not isinstance(point_of_interest, Point):
        point_of_interest = Point(point_of_interest[0], point_of_interest[1])
    
    # Use default or custom Manning's n ranges
    n_ranges = custom_n_ranges if custom_n_ranges is not None else create_manning_minmax_df()
    
    # Create timestamp for unique run identifier
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Initialize RAS project
    print(f"Initializing HEC-RAS project: {project_folder}")
    ras = init_ras_project(project_folder, "6.6")
    
    # Create output directory
    results_dir = Path(project_folder) / output_folder
    results_dir.mkdir(exist_ok=True)
    print(f"Results will be saved to: {results_dir}")
    
    # Verify template plan exists
    if template_plan not in ras.plan_df['plan_number'].values:
        raise ValueError(f"Template plan {template_plan} not found in project")
    
    # Get the geometry number for the template plan
    template_geom = ras.plan_df.loc[ras.plan_df['plan_number'] == template_plan, 'geometry_number'].values[0]
    print(f"\nTemplate plan: {template_plan} (Geometry: {template_geom})")
    
    # Get the geometry file path
    geom_path = ras.geom_df.loc[ras.geom_df['geom_number'] == template_geom, 'full_path'].values[0]
    
    # Get the original Manning's values
    original_baseoverrides = RasGeo.get_mannings_baseoverrides(geom_path)
    original_regionoverrides = RasGeo.get_mannings_regionoverrides(geom_path)
    
    # Check if regional overrides exist
    if original_regionoverrides.empty:
        print("No regional Manning's overrides found in the model")
        return None
    
    # If a specific region name is provided, filter the regional overrides
    if region_name is not None:
        region_mask = original_regionoverrides['Region Name'] == region_name
        if not region_mask.any():
            print(f"Region '{region_name}' not found in the model")
            available_regions = original_regionoverrides['Region Name'].unique()
            print(f"Available regions: {available_regions}")
            return None
        
        region_overrides = original_regionoverrides[region_mask].copy()
        print(f"\nAnalyzing sensitivity for region: {region_name}")
    else:
        region_overrides = original_regionoverrides.copy()
        print("\nAnalyzing sensitivity for all regions")
    
    # Get unique region tables
    region_tables = region_overrides['Table Number'].unique()
    print(f"Region tables: {region_tables}")
    
    # Analyze land cover statistics for the 2D mesh areas
    print("\nAnalyzing land cover statistics for the 2D mesh areas...")
    landcover_stats = analyze_mesh_land_cover_statistics(
        project_folder, 
        geom_number=template_geom
    )
    
    if landcover_stats is None:
        raise ValueError("Could not analyze land cover statistics")
    
    # Identify significant land uses (above threshold)
    significant_landuses = landcover_stats[landcover_stats['Percentage'] >= area_threshold].copy()
    significant_landuses = significant_landuses.sort_values('Percentage', ascending=False).reset_index(drop=True)
    
    if len(significant_landuses) == 0:
        print(f"No land uses found with coverage above {area_threshold}% threshold")
        return None
    
    print(f"\nFound {len(significant_landuses)} significant land uses (>= {area_threshold}% coverage):")
    print(significant_landuses[['Land Cover Type', 'Percentage', 'Current_n']])
    
    # Filter significant land uses to only those present in the region overrides
    region_landcover_types = set(region_overrides['Land Cover Name'].unique())
    filtered_landuses = significant_landuses[
        significant_landuses['Land Cover Type'].isin(region_landcover_types)
    ].copy()
    
    if len(filtered_landuses) == 0:
        print("None of the significant land uses are present in the regional overrides")
        return None
    
    print(f"\nSignificant land uses present in regional overrides:")
    print(filtered_landuses[['Land Cover Type', 'Percentage']])
    
    # Check if we'll exceed the plan limit
    current_plan_count = len(ras.plan_df)
    max_plans = 99  # HEC-RAS limit
    remaining_plans = max_plans - current_plan_count
    
    # Estimate the number of plans needed
    estimated_plan_count = 0
    
    # Create a table to store land use sensitivity information
    sensitivity_table = []
    
    for _, landuse in filtered_landuses.iterrows():
        land_cover = landuse['Land Cover Type']
        
        # Find this land cover in the region overrides
        for region_table in region_tables:
            # Create mask for this land cover and table
            mask = (region_overrides['Land Cover Name'] == land_cover) & \
                   (region_overrides['Table Number'] == region_table)
            
            if not mask.any():
                continue
                
            current_n = region_overrides.loc[mask, 'MainChannel'].values[0]
            
            # Find matching land cover in n_ranges
            match = n_ranges[n_ranges['Land Cover Name'] == land_cover]
            if match.empty:
                continue
                
            min_n = match['min_n'].values[0]
            max_n = match['max_n'].values[0]
            
            # Count values between min and max at interval spacing, excluding current value
            values = generate_sensitivity_values(min_n, max_n, current_n, interval)
            num_values = len(values)
            estimated_plan_count += num_values
            
            # Add to sensitivity table
            region_name = region_overrides.loc[mask, 'Region Name'].values[0] if 'Region Name' in region_overrides.columns else f"Table {region_table}"
            sensitivity_table.append({
                'Land Cover': land_cover,
                'Region': region_name,
                'Table': region_table,
                'Current n': current_n,
                'Min n': min_n,
                'Max n': max_n,
                'Test Values': num_values,
                'n Range': f"{min_n:.3f} - {max_n:.3f}"
            })
    
    # Print the sensitivity analysis table
    if sensitivity_table:
        print("\nSensitivity Analysis Plan:")
        print("-" * 80)
        print(f"{'Land Cover':<20} {'Region':<15} {'Current n':<10} {'n Range':<15} {'Test Values':<12}")
        print("-" * 80)
        for row in sensitivity_table:
            print(f"{row['Land Cover']:<20} {row['Region']:<15} {row['Current n']:<10.3f} {row['n Range']:<15} {row['Test Values']:<12}")
        print("-" * 80)
        print(f"Total estimated plans to be created: {estimated_plan_count}")
        print("-" * 80)
    if estimated_plan_count > remaining_plans:
        print(f"\nWARNING: This analysis would create approximately {estimated_plan_count} plans, but only {remaining_plans} more plans can be added (limit is 99)")
        print("Consider adjusting the following to reduce the number of plans:")
        print(f"1. Increase the area threshold (currently {area_threshold}%)")
        print(f"2. Increase the interval between test values (currently {interval})")
        print(f"3. Reduce the min/max ranges for land uses")
        print(f"4. Select fewer land uses to test")
        print(f"5. Specify a single region to test (currently {'specific region' if region_name else 'all regions'})")
        
        # Ask for confirmation to continue
        response = input("\nDo you want to continue anyway? (y/n): ")
        if response.lower() != 'y':
            print("Analysis canceled")
            return None
    
    # Store the current (template) plan as base scenario
    scenarios = [{
        'name': 'Template',
        'plan_number': template_plan,
        'geom_number': template_geom,
        'shortid': 'Template',
        'land_cover': None,
        'region_name': None,
        'table_number': None,
        'n_value': None,
        'description': "Original Manning's n Values"
    }]
    
    # Function to create a modified plan with adjusted Manning's n values for a specific land use in a region
    def create_modified_plan(land_cover, table_number, region_name, new_n_value):
        # Create a shortid based on land cover, region, and n value
        # Convert land cover name to code (e.g. "Open Water" -> "OW")
        lc_code = ''.join([word[0] for word in land_cover.split() if word[0].isalpha()])
        if not lc_code:
            lc_code = land_cover[:2]
        lc_code = lc_code.upper()
        
        # Convert region name to code
        rg_code = ''.join([word[0] for word in region_name.split() if word[0].isalpha()])
        if not rg_code:
            rg_code = region_name[:2]
        rg_code = rg_code.upper()
        
        # Format n value for shortid
        n_str = f"{new_n_value:.3f}".replace(".", "")
        shortid = f"R_{lc_code}_{rg_code}_{n_str}"
        
        print(f"\nCreating plan for '{land_cover}' in '{region_name}' with n = {new_n_value} (ShortID: {shortid})")
        
        # Clone the template plan
        new_plan_number = RasPlan.clone_plan(template_plan, new_plan_shortid=shortid)
        
        # Clone the template geometry
        new_geom_number = RasPlan.clone_geom(template_geom)
        
        # Set the new plan to use the new geometry
        RasPlan.set_geom(new_plan_number, new_geom_number)
        
        # Get the new geometry file path
        new_geom_path = ras.geom_df.loc[ras.geom_df['geom_number'] == new_geom_number, 'full_path'].values[0]
        
        # Copy base overrides unchanged
        RasGeo.set_mannings_baseoverrides(new_geom_path, original_baseoverrides)
        
        # Create modified region overrides
        modified_regionoverrides = original_regionoverrides.copy()
        
        # Update the Manning's n value for this specific land cover type in this region and table
        region_mask = (modified_regionoverrides['Land Cover Name'] == land_cover) & \
                     (modified_regionoverrides['Table Number'] == table_number) & \
                     (modified_regionoverrides['Region Name'] == region_name)
                     
        if region_mask.any():
            current_n = modified_regionoverrides.loc[region_mask, 'MainChannel'].values[0]
            print(f"  Changing '{land_cover}' in '{region_name}' (Table {table_number}) from {current_n:.4f} to {new_n_value:.4f}")
            modified_regionoverrides.loc[region_mask, 'MainChannel'] = new_n_value
        else:
            print(f"  Warning: Land cover '{land_cover}' not found in region '{region_name}' (Table {table_number})")
        
        # Apply the modified region overrides
        RasGeo.set_mannings_regionoverrides(new_geom_path, modified_regionoverrides)
        
        # Store scenario details
        return {
            'name': f"{land_cover}_{region_name}_{new_n_value:.3f}",
            'plan_number': new_plan_number,
            'geom_number': new_geom_number,
            'shortid': shortid,
            'land_cover': land_cover,
            'region_name': region_name,
            'table_number': table_number,
            'n_value': new_n_value,
            'description': f"Manning's n = {new_n_value:.3f} for {land_cover} in {region_name}"
        }
    
    # Create plans for each significant land use with varying n values
    all_plans_to_run = []
    
    for _, landuse in filtered_landuses.iterrows():
        land_cover = landuse['Land Cover Type']
        
        # Find matching land cover in n_ranges
        match = n_ranges[n_ranges['Land Cover Name'] == land_cover]
        
        if match.empty:
            print(f"Warning: No Manning's n range found for '{land_cover}'. Skipping.")
            continue
            
        min_n = match['min_n'].values[0]
        max_n = match['max_n'].values[0]
        
        # Process each region table for this land cover
        for region_table in region_tables:
            # Get all regions with this land cover in this table
            regions_mask = (region_overrides['Land Cover Name'] == land_cover) & \
                          (region_overrides['Table Number'] == region_table)
            
            if not regions_mask.any():
                continue
            
            # Get unique region names for this land cover and table
            unique_regions = region_overrides.loc[regions_mask, 'Region Name'].unique()
            
            for region in unique_regions:
                # If a specific region was requested, skip others
                if region_name is not None and region != region_name:
                    continue
                
                # Create mask for this specific combination
                specific_mask = (region_overrides['Land Cover Name'] == land_cover) & \
                               (region_overrides['Table Number'] == region_table) & \
                               (region_overrides['Region Name'] == region)
                
                if not specific_mask.any():
                    continue
                
                current_n = region_overrides.loc[specific_mask, 'MainChannel'].values[0]
                
                print(f"\nProcessing land cover: {land_cover} in region: {region} (Table {region_table})")
                print(f"  Current n: {current_n:.4f}")
                print(f"  Literature range: {min_n:.4f} to {max_n:.4f}")
                
                # Generate test values within the range, excluding the current value
                test_values = generate_sensitivity_values(min_n, max_n, current_n, interval)
                
                print(f"  Testing {len(test_values)} values: {[round(val, 3) for val in test_values]}")
                
                # Create a plan for each test value
                for n_value in test_values:
                    new_scenario = create_modified_plan(land_cover, region_table, region, n_value)
                    scenarios.append(new_scenario)
                    all_plans_to_run.append(new_scenario['plan_number'])
    
    # Save scenario information
    scenario_info = pd.DataFrame(scenarios)
    scenario_info_path = results_dir / "scenarios.csv"
    scenario_info.to_csv(scenario_info_path, index=False)
    print(f"\nScenario information saved to: {scenario_info_path}")
    
    # Run the plans (excluding the template which is already computed)
    plans_to_run = [plan for plan in all_plans_to_run if plan != template_plan]
    
    if not plans_to_run:
        print("No plans to run.")
        return {'scenarios': scenarios, 'output_folder': results_dir}
    
    print(f"\nRunning {len(plans_to_run)} plans in parallel...")
    execution_results = RasCmdr.compute_parallel(
        plan_number=plans_to_run,
        max_workers=max_workers,
        num_cores=num_cores,
        clear_geompre=True
    )
    
    print("\nExecution results:")
    for plan, success in execution_results.items():
        print(f"  Plan {plan}: {'Successful' if success else 'Failed'}")
    
    # If point of interest provided, extract and compare results
    if point_of_interest is not None:
        # Get geometry HDF path for cell identification
        geom_hdf_path = ras.geom_df.loc[ras.geom_df['geom_number'] == template_geom, 'hdf_path'].values[0]
        
        # Find the nearest mesh cell
        mesh_cells_gdf = HdfMesh.get_mesh_cell_points(geom_hdf_path)
        distances = mesh_cells_gdf.geometry.apply(lambda geom: geom.distance(point_of_interest))
        nearest_idx = distances.idxmin()
        mesh_cell_id = mesh_cells_gdf.loc[nearest_idx, 'cell_id']
        mesh_name = mesh_cells_gdf.loc[nearest_idx, 'mesh_name']
        
        print(f"\nNearest cell ID: {mesh_cell_id}")
        print(f"Distance: {distances[nearest_idx]:.2f} units")
        print(f"Mesh area: {mesh_name}")
        
        # Extract results for each scenario
        all_results = {}
        max_ws_values = []
        
        for scenario in scenarios:
            plan_number = scenario['plan_number']
            land_cover = scenario['land_cover']
            region_name = scenario['region_name']
            n_value = scenario['n_value']
            shortid = scenario['shortid']
            
            try:
                results_xr = HdfResultsMesh.get_mesh_cells_timeseries(plan_number)
                
                # Extract water surface data
                ws_data = results_xr[mesh_name]['Water Surface'].sel(cell_id=int(mesh_cell_id))
                
                # Convert to DataFrame
                ws_df = pd.DataFrame({
                    'time': ws_data.time.values,
                    'water_surface': ws_data.values
                })
                
                # Store results
                max_ws = ws_df['water_surface'].max()
                
                all_results[plan_number] = {
                    'scenario': scenario,
                    'df': ws_df,
                    'max_water_surface': max_ws
                }
                
                max_ws_values.append({
                    'plan_number': plan_number,
                    'shortid': shortid,
                    'land_cover': land_cover,
                    'region_name': region_name,
                    'n_value': n_value,
                    'max_water_surface': max_ws
                })
                
                print(f"  {shortid}: Max WSE = {max_ws:.2f}")
                
                # Save time series to CSV
                ws_df.to_csv(results_dir / f"timeseries_{shortid}.csv", index=False)
                
            except Exception as e:
                print(f"  Error extracting results for {shortid}: {str(e)}")
        
        # Create summary DataFrame
        if max_ws_values:
            max_ws_df = pd.DataFrame(max_ws_values)
            max_ws_df.to_csv(results_dir / "max_water_surface_summary.csv", index=False)
            
            # Create plots by land cover type and region
            land_cover_region_combinations = []
            
            for _, row in max_ws_df.iterrows():
                if row['land_cover'] is not None and row['region_name'] is not None:
                    combination = (row['land_cover'], row['region_name'])
                    if combination not in land_cover_region_combinations:
                        land_cover_region_combinations.append(combination)
            
            # Create sensitivity plots for each land cover + region combination
            for land_cover, region in land_cover_region_combinations:
                # Filter scenarios for this combination
                combo_scenarios = max_ws_df[
                    (max_ws_df['land_cover'] == land_cover) & 
                    (max_ws_df['region_name'] == region)
                ].copy()
                
                # Add the template scenario
                template_row = max_ws_df[max_ws_df['shortid'] == 'Template']
                if not template_row.empty:
                    combo_scenarios = pd.concat([template_row, combo_scenarios])
                
                if combo_scenarios.empty:
                    continue
                
                # Sort by n_value
                combo_scenarios = combo_scenarios.sort_values('n_value').reset_index(drop=True)
                
                # Create plot
                fig, ax = plt.subplots(figsize=(10, 6))
                ax.plot(combo_scenarios['n_value'], combo_scenarios['max_water_surface'], 
                         marker='o', linestyle='-', linewidth=2)
                
                # Add template point in a different color if it exists
                template_idx = combo_scenarios[combo_scenarios['shortid'] == 'Template'].index
                if not template_idx.empty:
                    ax.scatter(combo_scenarios.loc[template_idx, 'n_value'], 
                                combo_scenarios.loc[template_idx, 'max_water_surface'],
                                color='red', s=100, zorder=5, label='Template')
                
                # Add labels and title
                ax.set_xlabel(f"Manning's n for {land_cover} in {region}")
                ax.set_ylabel("Maximum Water Surface Elevation (ft)")
                ax.set_title(f"Sensitivity to {land_cover} Manning's n Value in {region}")
                ax.grid(True, linestyle='--', alpha=0.7)
                
                if not template_idx.empty:
                    ax.legend()
                
                # Save plot
                safe_lc = land_cover.replace(' ', '_').replace('/', '_')
                safe_rg = region.replace(' ', '_').replace('/', '_')
                plot_path = results_dir / f"sensitivity_{safe_lc}_{safe_rg}.png"
                plt.tight_layout()
                plt.savefig(plot_path)
                plt.close()
                print(f"Created sensitivity plot for {land_cover} in {region}")
            
            # Create time series comparison plots for each land cover + region combination
            for land_cover, region in land_cover_region_combinations:
                fig, ax = plt.subplots(figsize=(12, 6))
                
                # Get template results
                template_plan = scenarios[0]['plan_number']
                if template_plan in all_results:
                    template_df = all_results[template_plan]['df']
                    ax.plot(template_df['time'], template_df['water_surface'], 
                             color='black', linewidth=2, label='Template')
                
                # Filter scenarios for this combination
                combo_scenarios = [
                    s for s in scenarios 
                    if s['land_cover'] == land_cover and s['region_name'] == region
                ]
                
                if not combo_scenarios:
                    plt.close()
                    continue
                
                # Setup colormap for n values
                n_values = [s['n_value'] for s in combo_scenarios if s['n_value'] is not None]
                if not n_values:
                    plt.close()
                    continue
                    
                min_n = min(n_values)
                max_n = max(n_values)
                norm = plt.Normalize(min_n, max_n)
                cmap = plt.cm.viridis
                
                # Plot each scenario
                for scenario in combo_scenarios:
                    plan_number = scenario['plan_number']
                    n_value = scenario['n_value']
                    
                    if plan_number in all_results and n_value is not None:
                        df = all_results[plan_number]['df']
                        color = cmap(norm(n_value))
                        ax.plot(df['time'], df['water_surface'], color=color, 
                                 linewidth=1, alpha=0.7, label=f"n = {n_value:.3f}")
                
                # Add labels and title
                ax.set_xlabel("Time")
                ax.set_ylabel("Water Surface Elevation (ft)")
                ax.set_title(f"WSE Time Series for {land_cover} in {region}")
                ax.grid(True, linestyle='--', alpha=0.7)
                
                # Add colorbar
                sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
                sm.set_array([])
                cbar = plt.colorbar(sm, ax=ax)
                cbar.set_label(f"Manning's n for {land_cover}")
                
                # Save plot
                safe_lc = land_cover.replace(' ', '_').replace('/', '_')
                safe_rg = region.replace(' ', '_').replace('/', '_')
                plot_path = results_dir / f"timeseries_{safe_lc}_{safe_rg}.png"
                plt.tight_layout()
                plt.savefig(plot_path)
                plt.close()
                print(f"Created time series plot for {land_cover} in {region}")
    
    # Return results
    return {
        'scenarios': scenarios,
        'execution_results': execution_results if 'execution_results' in locals() else None,
        'results': all_results if 'all_results' in locals() else None,
        'max_ws_summary': max_ws_df if 'max_ws_df' in locals() else None,
        'significant_landuses': filtered_landuses,
        'output_folder': results_dir
    }

-----

-----

Engineer's Note: For regional overrides of the main channel would typically be made in bulk (see the previous example notebook that demonstrates bulk-varying mannings values), as land use types are based on satellite imagery that may not correlate well with main channel roughness.  We are individually varying by land use in this example, due to lack of available example models with large calibration regions.  Please note that this methodology is more applicable to large calibration regions within a mesh, not a main channel override.  For sensitivity to main channel overrides, use the bulk sensitivity for regional overrides approach, where all values are overridden, and granularity can be achieved by delineating multiple main channel regional overrides.

In [ ]:
# Example usage for Regional Overrides Sensitivity Analysis
# To run this, uncomment the code, adjust parameters as needed, and execute the cell


# Define project path and template plan (if not already defined)
# Assuming you've already extracted the example project
project_folder = Path(os.getcwd()) / "example_projects" / "BaldEagleCrkMulti2D"
template_plan = "03"  # Plan 03 has regional overrides

# Define a point of interest for result extraction
point_of_interest = (2081544, 365715)

# Run the regional sensitivity analysis
run_region_sensitivity_results = individual_landuse_sensitivity_region(
    project_folder=project_folder,
    template_plan=template_plan,
    point_of_interest=point_of_interest,
    area_threshold=10.0,  # Only analyze land uses covering at least 10% of the mesh area
    interval=0.02,       # Adjust interval to reduce the number of test values
    max_workers=2,
    num_cores=1,
    region_name="Main Channel",  # Specify a region or set to None for all regions
    output_folder="Regional_Landuse_Sensitivity"
)




In [ ]:
# Print summary information
if run_region_sensitivity_results:
    print("\nAnalysis complete! Results saved to:", run_region_sensitivity_results['output_folder'])
    if 'significant_landuses' in run_region_sensitivity_results:
        print("\nSignificant land uses analyzed in regions:")
        print(run_region_sensitivity_results['significant_landuses'][['Land Cover Type', 'Percentage']])

-----

# Example Regional Sensitivity from HEC Example Project BaldEagleCrkMulti2D, Plan 03

# Regional Sensitivity Results from the HEC Example Project BaldEagleCrkMulti2D, Plan 03: 

<!-- Cultivated Crops -->
![Time Series - Cultivated Crops](data/manning_img/regional_sensitivity/timeseries_Cultivated_Crops_Main_Channel.png)
![Mannings n Sensitivity - Cultivated Crops](data/manning_img/regional_sensitivity/sensitivity_Cultivated_Crops_Main_Channel.png)

<!-- Deciduous Forest -->
![Time Series - Deciduous Forest](data/manning_img/regional_sensitivity/timeseries_Deciduous_Forest_Main_Channel.png)
![Mannings n Sensitivity - Deciduous Forest](data/manning_img/regional_sensitivity/sensitivity_Deciduous_Forest_Main_Channel.png)

<!-- Developed High Intensity -->
![Time Series - Developed High Intensity](data/manning_img/regional_sensitivity/timeseries_Developed_High_Intensity_Main_Channel.png)
![Mannings n Sensitivity - Developed High Intensity](data/manning_img/regional_sensitivity/sensitivity_Developed_High_Intensity_Main_Channel.png)

<!-- Developed Low Intensity -->
![Time Series - Developed Low Intensity](data/manning_img/regional_sensitivity/timeseries_Developed_Low_Intensity_Main_Channel.png)
![Mannings n Sensitivity - Developed Low Intensity](data/manning_img/regional_sensitivity/sensitivity_Developed_Low_Intensity_Main_Channel.png)

<!-- Emergent Herbaceous Wetlands -->
![Time Series - Emergent Herbaceous Wetlands](data/manning_img/regional_sensitivity/timeseries_Emergent_Herbaceous_Wetlands_Main_Channel.png)
![Mannings n Sensitivity - Emergent Herbaceous Wetlands](data/manning_img/regional_sensitivity/sensitivity_Emergent_Herbaceous_Wetlands_Main_Channel.png)

<!-- Evergreen Forest -->
![Time Series - Evergreen Forest](data/manning_img/regional_sensitivity/timeseries_Evergreen_Forest_Main_Channel.png)
![Mannings n Sensitivity - Evergreen Forest](data/manning_img/regional_sensitivity/sensitivity_Evergreen_Forest_Main_Channel.png)

<!-- No Data -->
![Time Series - No Data](data/manning_img/regional_sensitivity/timeseries_NoData_Main_Channel.png)
![Mannings n Sensitivity - No Data](data/manning_img/regional_sensitivity/sensitivity_NoData_Main_Channel.png)

<!-- Pasture Hay -->
![Time Series - Pasture Hay](data/manning_img/regional_sensitivity/timeseries_Pasture_Hay_Main_Channel.png)
![Mannings n Sensitivity - Pasture Hay](data/manning_img/regional_sensitivity/sensitivity_Pasture_Hay_Main_Channel.png)

<!-- Woody Wetlands -->
![Time Series - Woody Wetlands](data/manning_img/regional_sensitivity/timeseries_Woody_Wetlands_Main_Channel.png)
![Mannings n Sensitivity - Woody Wetlands](data/manning_img/regional_sensitivity/sensitivity_Woody_Wetlands_Main_Channel.png)